In [27]:
import pandas as pd
import pymongo
import json


In [28]:
artist_name = "Novi Fosili"
artist_id = 'oOIOCzZV'
artist_image = "https://img.discogs.com/vmRqNB4olp7CK7_s530pFoiwfx4=/484x482/smart/filters:strip_icc():format(jpeg):mode_rgb():quality(90)/discogs-images/A-397578-1522620271-8991.jpeg.jpg"

In [29]:
## DB 
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
test_db = myclient["test_db"]
artists_col = test_db["artists"]

# artists_col.delete_many({})

# artist_dict = { "name": artist_name, "image": artist_image, "nouns": "", "verbs": "", "adjectives": "", "pronouns": "", "adverbs": ""}

# artist_schema = { 
#     "name": artist_name, 
#     "image": artist_image, 
#     "nouns": {"common": "", "proper": ""},
#     "verbs": {"main": "", "auxiliary": "", "modal": "", "copula": ""},
#     "adjectives": {"general": ""},
#     "pronouns": {"personal": "", "demonstrative": "", "indefinite": "", "possessive": "", "interrogative": "", "relative": "", "reflexive": ""}, 
#     "adverbs": {"general": ""}
#     }



artist_schema = { 
    "name": artist_name, 
    "image": artist_image, 
    "nouns": {"word": "", "repeats": "", "type": ""},
    "verbs": {"word": "", "repeats": "", "type": ""},
    "adjectives": {"word": "", "repeats": "", "type": ""},
    "pronouns": {"word": "", "repeats": "", "type": ""}, 
    "adverbs": {"word": "", "repeats": "", "type": ""},
    }


artists_col.insert_one(artist_schema)

In [30]:

df = pd.read_csv('lyrics.csv')
df = df.dropna()

df = df[df.Tags != "Z"] #remove punctionation (tag = Z)


In [40]:
#define which data to analyze

#by artist ID
df["toAnalyze"] = df["Song_ID"].str.find(artist_id) 
#df_old = df[df.toAnalyze == -1]
#df_old = df_old.drop(["toAnalyze"], axis=1)

# 
# 

In [41]:
df


,Artist,Song,Song_ID,Surface,Tags,Lemma,Entity,toAnalyze
0,Novi fosili,Ako se rastanemo,oOIOCzZV_0,ne,Qz,ne,O,0
1,Novi fosili,Ako se rastanemo,oOIOCzZV_0,govorim,Vmr1s,govoriti,O,0
2,Novi fosili,Ako se rastanemo,oOIOCzZV_0,s,Si,sa,O,0
3,Novi fosili,Ako se rastanemo,oOIOCzZV_0,tobom,Pp2-si,ti,O,0
4,Novi fosili,Ako se rastanemo,oOIOCzZV_0,i,Cc,i,O,0
...,...,...,...,...,...,...,...,...
8131,Novi fosili,Znam,oOIOCzZV_40,nebo,Ncnsn,nebo,O,0
8132,Novi fosili,Znam,oOIOCzZV_40,sve,Agpnsny,sav,O,0
8133,Novi fosili,Znam,oOIOCzZV_40,vidi,Vmr3s,vidjeti,O,0
8134,Novi fosili,Znam,oOIOCzZV_40,sve,Agpnsny,sav,O,0


In [32]:
df = df[df.toAnalyze >= 0]
df = df.reset_index()
df = df.drop(["index", "toAnalyze"], axis=1)


,Artist,Song,Song_ID,Surface,Tags,Lemma,Entity
0,Novi fosili,Ako se rastanemo,oOIOCzZV_0,ne,Qz,ne,O
1,Novi fosili,Ako se rastanemo,oOIOCzZV_0,govorim,Vmr1s,govoriti,O
2,Novi fosili,Ako se rastanemo,oOIOCzZV_0,s,Si,sa,O
3,Novi fosili,Ako se rastanemo,oOIOCzZV_0,tobom,Pp2-si,ti,O
4,Novi fosili,Ako se rastanemo,oOIOCzZV_0,i,Cc,i,O
...,...,...,...,...,...,...,...
8131,Novi fosili,Znam,oOIOCzZV_40,nebo,Ncnsn,nebo,O
8132,Novi fosili,Znam,oOIOCzZV_40,sve,Agpnsny,sav,O
8133,Novi fosili,Znam,oOIOCzZV_40,vidi,Vmr3s,vidjeti,O
8134,Novi fosili,Znam,oOIOCzZV_40,sve,Agpnsny,sav,O


In [33]:
wordType_dict = {"nouns": "N", "verbs": "V", "adjectives": "A", "pronouns": "P", "adverbs": "R"}

subType_dict = {
    "nouns": {"common": "Nc", "proper": "Np"}, 
    "verbs": {"main": "Vm", "auxiliary": "Va", "modal": "Vo", "copula": "Vc"},
    "adjectives": {"general": "Ag"},
    "pronouns": {"personal": "Pp", "demonstrative": "Pd", "indefinite": "Pi", "possessive": "Ps", "interrogative": "Pq", "relative": "Pr", "reflexive": "Px"}, 
    "adverbs": {"general": "Rg"}
    }

In [34]:
df_reduced = df[["Tags", "Lemma"]]

for wordType in wordType_dict:
    code = wordType_dict[wordType]
    words = df_reduced[df_reduced["Tags"].str.find(code) >= 0]
    all_types = pd.DataFrame(columns = ["word", "repeats", "type"])

    for subtype in subType_dict[wordType]:
        code_sub = subType_dict[wordType][subtype]

        words_sub = words[words["Tags"].str.find(code_sub) >= 0]     

        word_count = words_sub.Lemma.value_counts()
        word_count = pd.DataFrame(word_count)
        word_count = word_count.reset_index()
        word_count.columns = ["word", "repeats"]
        word_count["type"] = subtype
        
        word_count = word_count[word_count["repeats"]>5]
        
        all_types = all_types.append(word_count, ignore_index=True)
    
    all_types = all_types.sort_values(by=["repeats"], ascending=False)

    result = all_types.to_json(orient="records")
    parsed = json.loads(result)           
    
    newvalues = { "$set": { wordType: parsed } }
    artists_col.update_one({ "name": artist_name }, newvalues)

In [46]:
df_Entity = df[["Entity", "Lemma"]]
df_Entity

,Entity,Lemma
0,O,ne
1,O,govoriti
2,O,sa
3,O,ti
4,O,i
...,...,...
8131,O,nebo
8132,O,sav
8133,O,vidjeti
8134,O,sav


In [45]:
namedEntities = df_Entity[df_Entity["Entity"].str.find("O") = 0]


TypeError: '>' not supported between instances of 'str' and 'int'

In [36]:
# df_reduced = df[["Tags", "Lemma"]]

# for wordType in wordType_dict:
#     code = wordType_dict[wordType]
#     words = df_reduced[df_reduced["Tags"].str.find(code) >= 0]

#     for subtype in subType_dict[wordType]:
#         code_sub = subType_dict[wordType][subtype]
#         words_sub = words[words["Tags"].str.find(code_sub) >= 0]
        
#         word_count = words_sub.Lemma.value_counts()
#         word_count = pd.DataFrame(word_count)
#         word_count = word_count.reset_index()

#         word_count.columns = ["word", "repeats"]
#         word_count = word_count[word_count["repeats"]>5]

#         if word_count.size > 0:
#             result = word_count.to_json(orient="records")
#             parsed = json.loads(result)
#             # print(words_sub)
#             print(parsed[0])
        
            
#             newvalues = { "$set": { '.'.join([wordType, subtype]): parsed } }
#             # print(newvalues)
#             artists_col.update_one({ "name": artist_name }, newvalues)


In [37]:
df_reduced[df_reduced["Tags"].str.find("A") >= 0]


,Tags,Lemma
48,Agpmpny,sam
55,Agcfsly,tvoj
66,Agpmpny,nov
76,Agpfsny,drugačiji
79,Agpfsny,isti
...,...,...
8063,Agpnsny,sav
8065,Agpnsny,sav
8091,Agpmpny,dobar
8132,Agpnsny,sav


In [38]:
for x in artists_col.find({},{"name": 1}):
  print(x)


{'_id': ObjectId('5f382fd93b7be9a9dcc10879'), 'name': 'Severina'}
{'_id': ObjectId('5f3830b03b7be9a9dcc1087b'), 'name': 'Mate Mišo Kovač'}
{'_id': ObjectId('5f38310a3b7be9a9dcc1087d'), 'name': 'Novi Fosili'}


In [39]:
x = artists_col.find({"_id": '5f355d145d99f5672fdae024'})
print(x)